In [1]:
import os
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pymc_marketing import clv
os.chdir("../")
from src.data import (
    getDataset,
    ProcessData
)
from src.config import RawFeatures, RFM
from src.models import BetaGeoModel
from src.utils import get_customer_history_data, get_customer_whatif_data

In [2]:
import arviz as az
from arviz.labels import MapLabeller

In [3]:
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

In [4]:
dt_format_, _dt_format= '%d/%m/%Y %H:%M', '%Y-%m-%d %H:%M'
df_transaction = getDataset()
df_transaction[RawFeatures.TRANSACTION_DATE] = df_transaction[RawFeatures.TRANSACTION_DATE].apply(
                                                    lambda x: dt.datetime.strptime(x, dt_format_).strftime(_dt_format) if x==x else x)

In [5]:
df_transaction.head()

,Customer ID,InvoiceDate,Price,Quantity
0,13085.0,2009-12-01 07:45,6.95,12.0
1,13085.0,2009-12-01 07:45,6.75,12.0
2,13085.0,2009-12-01 07:45,6.75,12.0
3,13085.0,2009-12-01 07:45,2.10,48.0
4,13085.0,2009-12-01 07:45,1.25,24.0


In [6]:
data_inst = ProcessData(df_transaction, 'D', '2011-06-30')
data_summary = data_inst.model_data()
data_summary.head()

,frequency,recency,T,monetary_value
Customer ID,,,,
12346.0,7.0,400.0,725.0,11066.637143
12745.0,1.0,88.0,574.0,266.930000
12747.0,25.0,730.0,732.0,355.835600
12748.0,202.0,735.0,735.0,279.101436
12749.0,6.0,518.0,521.0,1010.743333


In [7]:
data_summary.shape

(3820, 4)

In [8]:
data_summary.describe().iloc[[1, 2, 3, 5, 7], :]

,frequency,recency,T,monetary_value
mean,6.490052,386.782461,526.574346,422.646958
std,10.721823,227.335755,201.691737,2771.151697
min,1.000000,1.000000,9.000000,3.900000
50%,3.000000,389.000000,592.000000,300.305000
max,202.000000,738.000000,738.000000,168469.600000


In [9]:
RFM()

RFM(max_recency=0.0, max_T=0.0, date_last_purchase=datetime.date(2023, 4, 29))

# Modeling


In [10]:
bgm = BetaGeoModel(data_summary)
bgm.fit()
bgm.fit_summary()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS using jitter+adapt_diag...


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 38 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.114,0.012,0.090,0.136,0.000,0.000,2071.0,2031.0,1.0
b,1.285,0.198,0.943,1.660,0.005,0.003,2005.0,1833.0,1.0
alpha,95.880,3.018,90.279,101.432,0.068,0.048,1978.0,2004.0,1.0
r,1.434,0.039,1.363,1.510,0.001,0.001,2073.0,2235.0,1.0


# Plots

In [11]:
"""az.plot_posterior(bgm.fit_result);"""

'az.plot_posterior(bgm.fit_result);'

In [12]:
"""clv.plot_probability_alive_matrix(bgm)
plt.show()"""

'clv.plot_probability_alive_matrix(bgm)\nplt.show()'

In [13]:
customer_id = 12749.0
n_period = 14
T_future_transac = 10

In [14]:
# T_, customer_history = get_customer_history_data(data_summary, customer_id, n_period)
T_, customer_history = get_customer_whatif_data(
                            data_summary, customer_id, n_period, T_future_transac
                            )
# customer_history

In [15]:
bgm.plot_probability_alive(
    customer_id,
    n_period,
    [1500, 700],
    T_future_transac
)

ValueError: 
    Invalid value of type 'builtins.bool' received for the 'marker' property of scatter
        Received value: True

    The 'marker' property is an instance of Marker
    that may be specified as:
      - An instance of :class:`plotly.graph_objs.scatter.Marker`
      - A dict of string/value properties that will be passed
        to the Marker constructor

        Supported dict properties:
            
            angle
                Sets the marker angle in respect to `angleref`.
            angleref
                Sets the reference for marker angle. With
                "previous", angle 0 points along the line from
                the previous point to this one. With "up",
                angle 0 points toward the top of the screen.
            anglesrc
                Sets the source reference on Chart Studio Cloud
                for `angle`.
            autocolorscale
                Determines whether the colorscale is a default
                palette (`auto

In [ ]:
(
alive_proba_study_time,
min_p_alive_,
max_p_alive_,
p_alive_xarray
) = bgm.probability_alive_features(
            T_,
            customer_history
        )
min_p_alive_

# Frontend Prep

```python
dt_format_, _dt_format= '%d/%m/%Y %H:%M', '%Y-%m-%d %H:%M'
df_transaction = getDataset()
df_transaction[RawFeatures.TRANSACTION_DATE] = df_transaction[RawFeatures.TRANSACTION_DATE].apply(
                                                    lambda x: dt.datetime.strptime(x, dt_format_).strftime(_dt_format) if x==x else x)

data_inst = ProcessData(df_transaction, 'D', '2011-06-30')
data_summary = data_inst.model_data()

bgm = BetaGeoModel(data_summary)
bgm.fit()
bgm.fit_summary()

customer_id = 14110.0
n_period = 14
T_future_transac = 7

bgm.plot_probability_alive(
    customer_id,
    n_period,
    [1500, 700],
    T_future_transac
)
```